In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

# importing after the general libraries to avoid initialization issues

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv

In [2]:
df_1 = pd.read_csv("中西_dataset.csv")
df_1.head()

/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/3642721045.py:1: DtypeWarning: Columns (47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv("中西_dataset.csv")


,ParkingSegmentID,year_val,month_val,day_val,half_hour_interval,weekday_x,district,avg_available_spots,TotalSpaces,lat,...,vapour_pressure_deficit,dew_point_2m,terrestrial_radiation_instant,latitude,longitude,hour,date,weekday_y,is_national_holiday,holiday_name
0,1421,2024,7,20,46,Saturday,中西區,1.0,3,22.9948,...,0.90,25.2,0.0,22.981,120.177,23,2024-07-20,Sat,False,NaN
1,1421,2024,7,24,40,Wednesday,中西區,1.0,3,22.9948,...,0.21,25.8,0.0,22.981,120.177,20,2024-07-24,Wed,False,NaN
2,1421,2024,7,24,18,Wednesday,中西區,1.0,3,22.9948,...,0.70,25.0,967.1,22.981,120.177,9,2024-07-24,Wed,False,NaN
3,1421,2024,8,21,9,Wednesday,中西區,2.0,3,22.9948,...,0.95,21.5,0.0,22.981,120.177,4,2024-08-21,Wed,False,NaN
4,1421,2024,9,9,5,Monday,中西區,1.0,3,22.9948,...,0.99,22.2,0.0,22.981,120.177,2,2024-09-09,Mon,False,NaN


In [3]:
df_2 = pd.read_csv("安平_dataset.csv")
df_2.head()

/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/1002289959.py:1: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv("安平_dataset.csv")


,ParkingSegmentID,year_val,month_val,day_val,half_hour_interval,weekday_x,district,avg_available_spots,TotalSpaces,lat,...,vapour_pressure_deficit,dew_point_2m,terrestrial_radiation_instant,latitude,longitude,hour,date,weekday_y,is_national_holiday,holiday_name
0,137,2025,3,28,47,Friday,安平區,46.500000,57,22.98953,...,1.17,15.5,0.0,22.981,120.177,23,2025-03-28,Fri,False,NaN
1,137,2024,7,11,26,Thursday,安平區,15.000000,57,22.98953,...,2.38,22.9,1290.3,22.981,120.177,13,2024-07-11,Thu,False,NaN
2,137,2025,3,28,17,Friday,安平區,29.833333,57,22.98953,...,1.10,20.0,639.6,22.981,120.177,8,2025-03-28,Fri,False,NaN
3,137,2024,7,11,22,Thursday,安平區,24.000000,57,22.98953,...,2.00,24.1,1278.5,22.981,120.177,11,2024-07-11,Thu,False,NaN
4,137,2025,3,28,10,Friday,安平區,49.000000,57,22.98953,...,0.65,20.8,0.0,22.981,120.177,5,2025-03-28,Fri,False,NaN


In [4]:
df = pd.concat([df_1, df_2])
df.head()
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 2892758 entries, 0 to 719269
Data columns (total 49 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   ParkingSegmentID               object 
 1   year_val                       int64  
 2   month_val                      int64  
 3   day_val                        int64  
 4   half_hour_interval             int64  
 5   weekday_x                      object 
 6   district                       object 
 7   avg_available_spots            float64
 8   TotalSpaces                    int64  
 9   lat                            float64
 10  lon                            float64
 11  firstHourFee                   int64  
 12  laterHourFee                   int64  
 13  datetime                       object 
 14  datetime_hour                  object 
 15  time                           object 
 16  precipitation                  float64
 17  apparent_temperature           float64
 18  relative

ParkingSegmentID                       0
year_val                               0
month_val                              0
day_val                                0
half_hour_interval                     0
weekday_x                              0
district                               0
avg_available_spots                    0
TotalSpaces                            0
lat                                    0
lon                                    0
firstHourFee                           0
laterHourFee                           0
datetime                               0
datetime_hour                          0
time                                   0
precipitation                          0
apparent_temperature                   0
relative_humidity_2m                   0
temperature_2m                         0
cloud_cover                            0
cloud_cover_low                        0
cloud_cover_mid                        0
cloud_cover_high                       0
visibility      

In [ ]:
df["ParkingSegmentID"] = df["ParkingSegmentID"].astype(str)
df.loc[
    (df["ParkingSegmentID"] == "1192") &
    (df["half_hour_interval"] == 20) &
    (df["month_val"] == 3) &
    (df["day_val"] == 8),
    "TotalSpaces"
] = 63
indices_to_drop = df[(df['ParkingSegmentID'] == "1131") & (df['year_val'] == 2025) & (df['month_val'] == 3) & (df['day_val'] == 27)].index
df.drop(indices_to_drop, inplace=True)
indices_to_drop = df[(df["year_val"] == 2025) & (df["month_val"] == 2) & (df["day_val"] == 20) & (df["half_hour_interval"] == 29) & (df["ParkingSegmentID"] == "54555420") & (df["TotalSpaces"] == 1036)].index
df.drop(indices_to_drop, inplace=True)
condition = df["avg_available_spots"] > df["TotalSpaces"]
# extract the values from df["TotalSpaces"] on the right side that meet the condition
values_to_assign = df.loc[condition, "TotalSpaces"].values
df.loc[condition, "avg_available_spots"] = values_to_assign

In [6]:
df.info()
df.describe()
df.isnull().sum()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 2892661 entries, 0 to 719269
Data columns (total 49 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   ParkingSegmentID               object 
 1   year_val                       int64  
 2   month_val                      int64  
 3   day_val                        int64  
 4   half_hour_interval             int64  
 5   weekday_x                      object 
 6   district                       object 
 7   avg_available_spots            float64
 8   TotalSpaces                    int64  
 9   lat                            float64
 10  lon                            float64
 11  firstHourFee                   int64  
 12  laterHourFee                   int64  
 13  datetime                       object 
 14  datetime_hour                  object 
 15  time                           object 
 16  precipitation                  float64
 17  apparent_temperature           float64
 18  relative

In [7]:
df["weekday"] = df["weekday_x"]

In [8]:

district_mapping = {'中西區': 0, '安平區': 1}
df['district'] = df['district'].map(district_mapping)

In [9]:
df["datetime_2"] = (
    df["year_val"].astype(str) + "_" +
    df["month_val"].astype(str) + "_" +
    df["day_val"].astype(str) + "_" +
    df["half_hour_interval"].astype(str)
)

In [10]:
pd.set_option('display.max_columns', None)
weekday_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3,
                       'Friday': 4, 'Saturday': 5, 'Sunday': 6}
df['weekday_num'] = df['weekday'].map(weekday_mapping)
df['wind_direction_10m_sin'] = np.sin(np.deg2rad(df['wind_direction_10m']))
df['wind_direction_10m_cos'] = np.cos(np.deg2rad(df['wind_direction_10m']))
df['wind_direction_120m_sin'] = np.sin(np.deg2rad(df['wind_direction_120m']))
df['wind_direction_120m_cos'] = np.cos(np.deg2rad(df['wind_direction_120m']))
df['wind_direction_80m_sin'] = np.sin(np.deg2rad(df['wind_direction_80m']))
df['wind_direction_80m_cos'] = np.cos(np.deg2rad(df['wind_direction_80m']))
df.drop(columns=["wind_direction_10m","wind_direction_120m","wind_direction_80m"],inplace=True)

In [ ]:

df_final = df.drop(columns=[         
    'weekday',        
    'weekday_y',"date","latitude","longitude","datetime_hour","time", "weekday_x","holiday_name",
])

In [12]:
df_final

,ParkingSegmentID,year_val,month_val,day_val,half_hour_interval,district,avg_available_spots,TotalSpaces,lat,lon,firstHourFee,laterHourFee,datetime,precipitation,apparent_temperature,relative_humidity_2m,temperature_2m,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,temperature_80m,temperature_120m,showers,pressure_msl,surface_pressure,wind_speed_80m,wind_speed_120m,wind_gusts_10m,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,dew_point_2m,terrestrial_radiation_instant,hour,is_national_holiday,datetime_2,weekday_num,wind_direction_10m_sin,wind_direction_10m_cos,wind_direction_120m_sin,wind_direction_120m_cos,wind_direction_80m_sin,wind_direction_80m_cos
0,1421,2024,7,20,46,0,1.000000,3,22.99480,120.19519,0,0,2024-07-20 23:00:00,0.8,35.4,78,29.4,15,0,0,10,24140.0,5.8,28.7,28.6,0.8,1010.1,1009.5,7.4,7.7,7.9,0.0,0.01,0.90,25.2,0.0,23,False,2024_7_20_46,5,-0.069756,-0.997564,-0.422618,-0.906308,-0.390731,-0.920505
1,1421,2024,7,24,40,0,1.000000,3,22.99480,120.19519,0,0,2024-07-24 20:00:00,15.2,27.6,94,26.9,100,100,100,100,2300.0,45.1,26.5,26.4,1.2,985.5,984.9,72.7,77.2,74.5,0.0,0.03,0.21,25.8,0.0,20,False,2024_7_24_40,2,-0.707107,0.707107,-0.642788,0.766044,-0.656059,0.754710
2,1421,2024,7,24,18,0,1.000000,3,22.99480,120.19519,20,20,2024-07-24 09:00:00,0.8,28.5,82,28.4,100,100,100,100,9660.0,45.3,27.9,27.7,0.8,989.1,988.5,68.6,72.6,66.2,0.0,0.15,0.70,25.0,967.1,9,False,2024_7_24_18,2,-0.358368,0.933580,-0.309017,0.951057,-0.309017,0.951057
3,1421,2024,8,21,9,0,2.000000,3,22.99480,120.19519,0,0,2024-08-21 04:30:00,0.0,30.0,73,26.8,99,0,90,90,24140.0,10.1,26.8,26.8,0.0,1010.4,1009.8,17.1,18.9,19.4,0.0,0.04,0.95,21.5,0.0,4,False,2024_8_21_9,2,0.573576,-0.819152,0.406737,-0.913545,0.469472,-0.882948
4,1421,2024,9,9,5,0,1.000000,3,22.99480,120.19519,0,0,2024-09-09 02:30:00,0.0,31.5,73,27.5,100,0,5,100,24140.0,7.1,26.7,26.5,0.0,1006.8,1006.2,8.7,8.8,8.3,0.0,0.02,0.99,22.2,0.0,2,False,2024_9_9_5,0,0.754710,-0.656059,0.707107,-0.707107,0.707107,-0.707107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719265,89053916,2025,5,9,47,1,103.000000,160,22.99884,120.16250,60,60,2025-05-09 23:30:00,0.0,33.3,57,30.4,100,0,10,100,24140.0,9.5,29.1,28.9,0.0,1009.5,1008.9,12.1,12.8,12.6,-0.0,0.07,1.86,20.9,0.0,23,False,2025_5_9_47,4,0.156434,-0.987688,0.207912,-0.978148,0.207912,-0.978148
719266,89053916,2025,5,9,25,1,84.333333,160,22.99884,120.16250,60,60,2025-05-09 12:30:00,0.0,36.7,51,33.5,100,0,5,100,24140.0,23.4,30.7,30.4,0.0,1009.1,1008.5,29.3,30.2,24.1,0.0,0.75,2.53,22.0,1336.1,12,False,2025_5_9_25,4,-0.422618,-0.906308,-0.358368,-0.933580,-0.358368,-0.933580
719267,89053916,2025,5,9,5,1,107.000000,160,22.99884,120.16250,60,60,2025-05-09 02:30:00,0.0,32.4,62,29.4,100,0,0,100,24140.0,10.6,28.2,28.0,0.0,1007.9,1007.3,14.9,15.6,16.6,-0.0,0.07,1.56,21.4,0.0,2,False,2025_5_9_5,4,0.173648,-0.984808,0.156434,-0.987688,0.139173,-0.990268
719268,89053916,2025,5,9,9,1,107.000000,160,22.99884,120.16250,60,60,2025-05-09 04:30:00,0.0,31.1,62,29.0,100,0,0,100,24140.0,15.6,27.9,27.7,0.0,1007.7,1007.1,22.6,24.1,27.4,0.0,0.10,1.52,21.0,0.0,4,False,2025_5_9_9,4,0.139173,-0.990268,0.139173,-0.990268,0.139173,-0.990268


In [ ]:
def add_cyclical_features(df):
    # month (1–12 → 12 periods)
    
    # date (1–31 → 31 periods)
    df['day_sin'] = np.sin(2 * np.pi * df['day_val'] / 31)
    df['day_cos'] = np.cos(2 * np.pi * df['day_val'] / 31)
    
    # half-hour intervals (0–47 → 48 periods)
    df['half_hour_sin'] = np.sin(2 * np.pi * df['half_hour_interval'] / 48)
    df['half_hour_cos'] = np.cos(2 * np.pi * df['half_hour_interval'] / 48)
    
    return df

df_final = add_cyclical_features(df_final)
df_final = pd.get_dummies(df_final, columns=['month_val', 'weekday_num'])
df_final["day_off"] = df_final["is_national_holiday"] | df_final["weekday_num_6"] | df_final["weekday_num_5"]
df_final.drop(columns=["is_national_holiday"], inplace = True)

In [14]:
df_final["datetime"] = pd.to_datetime(df_final["datetime"])
df_train = df_final[df_final["datetime"] < "2025-01-23"]
df_val = df_final[(df_final["datetime"] >= "2025-01-23") & (df_final["datetime"] < "2025-03-23")]
df_test = df_final[df_final["datetime"] >= "2025-03-23"]
df_train.drop(columns=["datetime"], inplace=True)
df_val.drop(columns=["datetime"], inplace=True)
df_test.drop(columns=["datetime"], inplace=True)
df_train.head()
df_val.head()
df_test.head()

/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/1851175704.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=["datetime"], inplace=True)
/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/1851175704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val.drop(columns=["datetime"], inplace=True)
/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/1851175704.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

,ParkingSegmentID,year_val,day_val,half_hour_interval,district,avg_available_spots,TotalSpaces,lat,lon,firstHourFee,laterHourFee,precipitation,apparent_temperature,relative_humidity_2m,temperature_2m,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,temperature_80m,temperature_120m,showers,pressure_msl,surface_pressure,wind_speed_80m,wind_speed_120m,wind_gusts_10m,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,dew_point_2m,terrestrial_radiation_instant,hour,datetime_2,wind_direction_10m_sin,wind_direction_10m_cos,wind_direction_120m_sin,wind_direction_120m_cos,wind_direction_80m_sin,wind_direction_80m_cos,day_sin,day_cos,half_hour_sin,half_hour_cos,month_val_1,month_val_2,month_val_3,month_val_4,month_val_5,month_val_6,month_val_7,month_val_8,month_val_9,month_val_10,month_val_11,month_val_12,weekday_num_0,weekday_num_1,weekday_num_2,weekday_num_3,weekday_num_4,weekday_num_5,weekday_num_6,day_off
66246,1421,2025,27,5,0,1.0,3,22.99480,120.19519,0,0,0.0,26.7,71,24.7,0,0,0,0,24140.0,9.6,23.4,23.2,0.0,1013.6,1013.0,11.3,11.9,8.6,-0.0,0.03,0.90,19.1,0.0,2,2025_4_27_5,0.224951,0.974370,0.156434,0.987688,0.156434,0.987688,-0.724793,0.688967,0.608761,0.793353,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
66247,1421,2025,28,40,0,1.0,3,22.99480,120.19519,0,0,0.0,28.1,62,27.4,0,0,0,0,24140.0,19.5,25.9,25.7,0.0,1011.2,1010.6,27.0,27.9,24.1,-0.0,0.11,1.39,19.5,0.0,20,2025_4_28_40,0.034899,0.999391,0.034899,0.999391,0.034899,0.999391,-0.571268,0.820763,-0.866025,0.500000,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
66250,1421,2025,4,20,0,1.0,3,22.99480,120.19519,20,20,0.0,36.2,49,31.9,83,0,0,82,24140.0,6.5,29.4,29.2,0.0,1009.3,1008.7,6.1,6.3,5.0,0.0,0.61,2.41,19.9,1187.1,10,2025_5_4_20,-0.829038,0.559193,-0.766044,0.642788,-0.766044,0.642788,0.724793,0.688967,0.500000,-0.866025,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
66256,181,2025,29,5,0,28.5,51,22.98488,120.20178,0,0,0.0,27.9,63,26.2,7,0,0,5,24140.0,9.8,24.9,24.7,0.0,1010.3,1009.7,14.4,15.2,11.2,-0.0,0.05,1.26,18.6,0.0,2,2025_4_29_5,-0.469472,-0.882948,-0.529919,-0.848048,-0.529919,-0.848048,-0.394356,0.918958,0.608761,0.793353,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
66261,181,2025,6,47,0,30.0,51,22.98488,120.20178,0,0,0.0,32.7,67,28.8,23,8,0,14,24140.0,6.9,27.3,27.1,0.0,1009.3,1008.7,7.8,7.9,7.6,-0.0,0.03,1.30,22.1,0.0,23,2025_5_6_47,-0.469472,-0.882948,-0.374607,-0.927184,-0.374607,-0.927184,0.937752,0.347305,-0.130526,0.991445,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [15]:
df_train['available_percent'] = df_train["avg_available_spots"]
df_val['available_percent'] = df_val["avg_available_spots"]
df_test['available_percent'] = df_test["avg_available_spots"]
df_train.drop(columns=["avg_available_spots"], inplace=True)
df_val.drop(columns=["avg_available_spots"], inplace=True)
df_test.drop(columns=["avg_available_spots"], inplace=True)

/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/3745555813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['available_percent'] = df_train["avg_available_spots"]
/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/3745555813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['available_percent'] = df_val["avg_available_spots"]
/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/3745555813.py:3: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
print(len(df_train.loc[df_train['datetime_2'] == "2024_1_23_2", 'ParkingSegmentID'].unique()))
print(len(df_train['ParkingSegmentID'].unique()))
print(len(df_test['ParkingSegmentID'].unique()))
# retrieve all parking lot IDs from the training set
train_seg_id = set(df_train.loc[df_train['datetime_2'] == "2024_1_23_2", 'ParkingSegmentID'].unique())

# filter validation/test sets to retain only the IDs present in the training set
df_train = df_train[df_train['ParkingSegmentID'].isin(train_seg_id)].copy()
df_val = df_val[df_val['ParkingSegmentID'].isin(train_seg_id)].copy()
df_test = df_test[df_test['ParkingSegmentID'].isin(train_seg_id)].copy()

print("val 停車場數量:", len(df_val['ParkingSegmentID'].unique()))
print("test 停車場數量:", len(df_test['ParkingSegmentID'].unique()))

147
152
157
val 停車場數量: 147
test 停車場數量: 147


In [17]:
df_train.columns

Index(['ParkingSegmentID', 'year_val', 'day_val', 'half_hour_interval',
       'district', 'TotalSpaces', 'lat', 'lon', 'firstHourFee', 'laterHourFee',
       'precipitation', 'apparent_temperature', 'relative_humidity_2m',
       'temperature_2m', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid',
       'cloud_cover_high', 'visibility', 'wind_speed_10m', 'temperature_80m',
       'temperature_120m', 'showers', 'pressure_msl', 'surface_pressure',
       'wind_speed_80m', 'wind_speed_120m', 'wind_gusts_10m',
       'evapotranspiration', 'et0_fao_evapotranspiration',
       'vapour_pressure_deficit', 'dew_point_2m',
       'terrestrial_radiation_instant', 'hour', 'datetime_2',
       'wind_direction_10m_sin', 'wind_direction_10m_cos',
       'wind_direction_120m_sin', 'wind_direction_120m_cos',
       'wind_direction_80m_sin', 'wind_direction_80m_cos', 'day_sin',
       'day_cos', 'half_hour_sin', 'half_hour_cos', 'month_val_1',
       'month_val_2', 'month_val_3', 'month_val_4', 'mon

In [18]:
spatial_cols = ['lat', 'lon'
                   ]
temporal_cols = [
  'TotalSpaces',
 'firstHourFee',
 'relative_humidity_2m',
 'et0_fao_evapotranspiration',
 'day_sin',
 'half_hour_sin',
 'half_hour_cos',
 'weekday_num_5',
 'weekday_num_6']
spatial_cols_to_scale = ['lat', 'lon']
temporal_cols_to_scale = [
                            'TotalSpaces',
 'firstHourFee',
 'relative_humidity_2m',
 'et0_fao_evapotranspiration',
]

In [ ]:
def build_spatial_df(df, spatial_cols):
    # return the static feature table deduplicated by ParkingSegmentID
    spatial_df = (
        df.groupby('ParkingSegmentID')[spatial_cols]
          .first()           
          .reset_index()
    )
    return spatial_df

spatial_df = build_spatial_df(df_train, spatial_cols)   
N = len(spatial_df)

In [ ]:
from collections import defaultdict
import numpy as np
import torch
test_time_key =[]
def prepare_full_timeslice_batches(df, spatial_df,
                                   temporal_cols, target_col='available_percent'):
    """
    Only keep the slices where **all N nodes are present** at that time point.
    Return batches: List[Dict], where each batch includes:
    temporal_features : [N, d_t] numpy array
    target            : [N] numpy array
    time_key          : time index (optional)
    """
    # create node ordering → to facilitate alignment with the order in spatial_df
    node_order = spatial_df['ParkingSegmentID'].tolist()
    node_idx_map = {pid: idx for idx, pid in enumerate(node_order)}
    N = len(node_order)
    batches = []
    # group by time step
    for time_key, g in df.groupby('datetime_2'):
        if len(g) != N:
            # incomplete data at this time point, skip directly
            continue
        # align according to node_order
        g_sorted = g.set_index('ParkingSegmentID').loc[node_order]
        assert len(g_sorted) == N, f"time {time_key} 節點遺失！"
        
        t_mat = g_sorted[temporal_cols].to_numpy(dtype=np.float32)                  # [N, d_t]
        tgt   = g_sorted[target_col].to_numpy(dtype=np.float32)                     # [N]
        test_time_key.append(time_key)
        batches.append({
            'time_key': time_key,
            'temporal_features': t_mat,
            'target': tgt,

        })
    return batches

In [ ]:

import matplotlib.pyplot as plt

# count the number of data records for each parking lot
count_per_parking = df_train['ParkingSegmentID'].value_counts()

# then count how many parking lots have exactly n data records
count_of_counts = count_per_parking.value_counts().sort_index()

plt.figure(figsize=(10,6))
plt.bar(count_of_counts.index, count_of_counts.values, width=1.0, edgecolor='black')
plt.xlabel('每個停車場的資料筆數')
plt.ylabel('有這麼多資料的停車場數量')
plt.title('停車場資料筆數分布')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

# create scaler
spatial_scaler   = {c: StandardScaler().fit(df_train[[c]]) for c in spatial_cols_to_scale}
temporal_scaler  = {c: StandardScaler().fit(df_train[[c]]) for c in temporal_cols_to_scale}

def scale_dataframe(df):
    df_scaled = df.copy()
    
    for c in spatial_cols_to_scale:
        df_scaled.loc[:, c] = spatial_scaler[c].transform(df_scaled[[c]]).ravel()
    for c in temporal_cols_to_scale:
        df_scaled.loc[:, c] = temporal_scaler[c].transform(df_scaled[[c]]).ravel()
    return df_scaled

# ------------- generate batches -------------
df_train_scaled = scale_dataframe(df_train)
df_val_scaled   = scale_dataframe(df_val)
df_test_scaled  = scale_dataframe(df_test)

train_batches = prepare_full_timeslice_batches(df_train_scaled,
                                               spatial_df,
                                               temporal_cols)

val_batches   = prepare_full_timeslice_batches(df_val_scaled,
                                               spatial_df,
                                               temporal_cols)
test_time_key =[]
test_batches  = prepare_full_timeslice_batches(df_test_scaled,
                                               spatial_df,
                                               temporal_cols)

/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/2811152063.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.36874861 -0.36874861 -0.36874861 ... -0.19343778 -0.19343778
 -0.19343778]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_scaled.loc[:, c] = temporal_scaler[c].transform(df_scaled[[c]]).ravel()
/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/2811152063.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.09947748 -1.09947748  0.76159863 ...  0.76159863  0.76159863
  0.76159863]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_scaled.loc[:, c] = temporal_scaler[c].transform(df_scaled[[c]]).ravel()
/var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/ipykernel_34315/2811152063.py:13: FutureWarning: Setting an i

In [23]:
node_order = spatial_df['ParkingSegmentID'].tolist()

In [24]:
df_test_out = df_test.loc[(df_test['datetime_2'].isin(test_time_key)) & (df_test['ParkingSegmentID'].isin(node_order))].drop(columns=['datetime_2','day_val'])

In [25]:
df_test_out['avg_available_spots'] = df_test_out['available_percent']
df_test_out.drop(columns=['half_hour_interval','hour','available_percent'],inplace=True)
df_test_out.to_csv('df_test_out.csv',index=False)


In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def build_spatial_graph(spatial_df, k=8):
    coords = spatial_df[['lat', 'lon']].values
    N = len(coords)
    
    nn_model = NearestNeighbors(n_neighbors=min(k+1, N))
    nn_model.fit(coords)
    distances, indices = nn_model.kneighbors(coords)
    
    # create edges and weights
    edge_dict = dict()
    for i in range(N):
        for j in range(1, min(k+1, N)):
            neighbor_idx = indices[i, j]
            if i != neighbor_idx:
                edge = tuple(sorted((i, neighbor_idx)))
                # use a Gaussian kernel for the weights
                distance = distances[i, j]
                weight = 1/distance
                # if the edge already exists, take the average of the weights
                if edge in edge_dict:
                    edge_dict[edge] = (edge_dict[edge] + weight) / 2
                else:
                    edge_dict[edge] = weight

    src, dst = zip(*edge_dict.keys())
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    edge_weight = torch.tensor(list(edge_dict.values()), dtype=torch.float)
    
    return edge_index, edge_weight

edge_index, edge_weight = build_spatial_graph(spatial_df, k=8)

In [ ]:
class SpatioTemporalGNN(torch.nn.Module):
    def __init__(self, spatial_features, temporal_features, hidden_dim=64, dropout=0.0, activation=torch.nn.ReLU()):
        super().__init__()
        
        # spatial features encoder
        self.spatial_encoder = torch.nn.Sequential(
            torch.nn.Linear(spatial_features, hidden_dim),
            activation,
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_dim, hidden_dim)
        )
        
        # temporal features encoder
        self.temporal_encoder = torch.nn.Sequential(
            torch.nn.Linear(temporal_features, hidden_dim),
            activation,
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_dim, hidden_dim)
        )
        
        # GNN layers
        self.conv1 = GCNConv(hidden_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        
        # prediction layer
        self.predictor = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim // 2),
            activation,
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_dim // 2, 1)
        )
        self.gate_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, hidden_dim),
            torch.nn.Sigmoid()
        )
            
    def forward(self, spatial_x, temporal_x, edge_index, edge_weight):
        # encode spatial features
        spatial_emb = self.spatial_encoder(spatial_x)
        
        # encode temporal features
        temporal_emb = self.temporal_encoder(temporal_x)
        # merge features
        concat_feat = torch.cat([spatial_emb, temporal_emb], dim=1)
        gate = self.gate_layer(concat_feat)  # [N, hidden_dim]
        x = gate * spatial_emb + (1 - gate) * temporal_emb 
                # GNN process
        x1 = F.relu(self.conv1(x, edge_index, edge_weight))
        x2 = F.relu(self.conv2(x1, edge_index, edge_weight)) + x1
        
        # predict
        return self.predictor(x)
def run_epoch(batches, model, spatial_x,
              edge_index, edge_attr,
              optimizer=None,
              batch_size=8,
              collect_pred=False):

    train_mode = optimizer is not None
    model.train() if train_mode else model.eval()

    total_loss, total_count = 0.0, 0
    y_true_all, y_pred_all = [], []

    for i in range(0, len(batches), batch_size):
        slice_batch = batches[i : i + batch_size]

        temporal_stack = torch.stack([
            torch.from_numpy(b['temporal_features'])
            for b in slice_batch
        ]).to(spatial_x.device, dtype=spatial_x.dtype)

        target_stack = torch.stack([
            torch.from_numpy(b['target']).unsqueeze(1)
            for b in slice_batch
        ]).to(spatial_x.device, dtype=torch.float32)

        batch_loss = 0.0
        batch_count = 0
        preds = []
        for t_feat, tgt in zip(temporal_stack, target_stack):
            out = model(spatial_x, t_feat, edge_index, edge_attr)
            per_node_loss = F.mse_loss(out, tgt, reduction='sum')
            batch_loss += per_node_loss
            batch_count += out.numel()
            preds.append(out)

        if train_mode:
            optimizer.zero_grad()
            (batch_loss / batch_count).backward()  # use a tensor to perform backward
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        if collect_pred:
            y_true_all.append(target_stack.detach().cpu())
            y_pred_all.append(torch.stack(preds).detach().cpu())
        total_loss += batch_loss.item()
        total_count += batch_count

    mean_loss = total_loss / total_count

    if collect_pred:
        y_true_all = torch.cat(y_true_all).numpy().flatten()
        y_pred_all = torch.cat(y_pred_all).numpy().flatten()
        return mean_loss, y_true_all, y_pred_all
    else:
        return mean_loss, None, None

In [28]:
model = SpatioTemporalGNN(
        spatial_features=len(spatial_cols),
        temporal_features=len(temporal_cols),
        hidden_dim=64,
        dropout=0,
        activation=torch.nn.ReLU()
    )
device = torch.device('cpu')

In [29]:
spatial_df

,ParkingSegmentID,lat,lon
0,1001,22.99728,120.17506
1,1002,22.99443,120.18191
2,101,22.98781,120.20494
3,1011,22.99679,120.17929
4,1071,22.98840,120.19851
...,...,...,...
142,911,22.99547,120.18517
143,92,22.98643,120.20818
144,981,22.99706,120.17182
145,982,22.98248,120.16979


In [30]:
from tqdm import tqdm
# static tensors
spatial_x  = torch.from_numpy(spatial_df[spatial_cols].to_numpy(dtype=np.float64)).to(device).float()
edge_index, edge_weight = build_spatial_graph(spatial_df, k=8)
edge_index  = edge_index.to(device)
edge_weight = edge_weight.to(device)

In [ ]:

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

best_val, patience, wait = float('inf'), 10, 0   # stop if there is no improvement for 10 consecutive trials
for epoch in tqdm(range(200)):
    train_loss, _, _ = run_epoch(
        train_batches, model, spatial_x,
        edge_index, edge_weight,
        optimizer=optimizer, batch_size=64)

    val_loss, yv, pv = run_epoch(
        val_batches,   model, spatial_x,
        edge_index, edge_weight,
        optimizer=None, batch_size=64, collect_pred=True)
    r2 = 1 - np.sum((yv - pv)**2) / np.sum((yv - yv.mean())**2)
    print(f"Epoch {epoch:02d} | train {train_loss:.4f} | val {val_loss:.4f} | R² {r2:5.3f}")

    # ---- Early stopping ----
    if val_loss < best_val - 1e-4:
        best_val, wait = val_loss, 0
        torch.save(model.state_dict(), "best_gnn_4.pt")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered!")
            break

  0%|          | 1/200 [00:10<35:25, 10.68s/it]

Epoch 00 | train 3473.6795 | val 2763.4116 | R² 0.320


  1%|          | 2/200 [00:21<34:52, 10.57s/it]

Epoch 01 | train 1822.8451 | val 1395.4665 | R² 0.657


  2%|▏         | 3/200 [00:30<33:31, 10.21s/it]

...

Epoch 71 | train 308.8591 | val 337.0444 | R² 0.917


 36%|███▋      | 73/200 [15:26<28:07, 13.28s/it]

Epoch 72 | train 327.7040 | val 328.9278 | R² 0.919


 36%|███▋      | 73/200 [15:40<27:15, 12.88s/it]

Epoch 73 | train 313.8606 | val 275.9366 | R² 0.932
Early stopping triggered!


In [ ]:
import optuna
# expand the search space in Optuna
def objective(trial):
    hidden_dim = trial.suggest_categorical('hidden_dim', [64, 128, 256])
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8,64,256, 512, 1024])
    k = trial.suggest_int('k', 4, 12, step=2)
    activation = torch.nn.ReLU()
    # reconstruct spatial graph
    edge_index, edge_weight = build_spatial_graph(spatial_df, k=k)
    edge_index = edge_index.to(device)
    edge_weight = edge_weight.to(device)
    model = SpatioTemporalGNN(
        spatial_features=len(spatial_cols),
        temporal_features=len(temporal_cols),
        hidden_dim=hidden_dim,
        dropout=dropout,
        activation=activation
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(f"Trial {trial.number} params: {hidden_dim}, {dropout}, {learning_rate}, {batch_size}, {k}")
    for epoch in range(10):
        print(f"Trial {trial.number} Epoch {epoch} start")
        run_epoch(train_batches, model, spatial_x, edge_index, edge_weight, optimizer, batch_size=batch_size)
        print(f"Trial {trial.number} Epoch {epoch} end")
    val_loss, yv, pv = run_epoch(val_batches, model, spatial_x, edge_index, edge_weight, optimizer=None, batch_size=batch_size, collect_pred=True)
    r2 = 1 - np.sum((yv - pv)**2) / np.sum((yv - yv.mean())**2)
    return val_loss

device = torch.device('cpu')
print('使用裝置:', device)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print('Best params:', study.best_params)
print('Best value:', study.best_value)

/Users/yhk/miniforge3/envs/gnn_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-01 22:25:47,693] A new study created in memory with name: no-name-55ff7644-f1d5-4079-bdb6-5cd91aa10d48


使用裝置: cpu
Trial 0 params: 128, 0.2366351608948103, 0.0003521826323511964, 64, 8
Trial 0 Epoch 0 start
Trial 0 Epoch 0 end
Trial 0 Epoch 1 start
Trial 0 Epoch 1 end
Trial 0 Epoch 2 start
Trial 0 Epoch 2 end
Trial 0 Epoch 3 start
Trial 0 Epoch 3 end
Trial 0 Epoch 4 start
Trial 0 Epoch 4 end
Trial 0 Epoch 5 start
Trial 0 Epoch 5 end
Trial 0 Epoch 6 start
Trial 0 Epoch 6 end
Trial 0 Epoch 7 start
Trial 0 Epoch 7 end
Trial 0 Epoch 8 start
Trial 0 Epoch 8 end
Trial 0 Epoch 9 start
Trial 0 Epoch 9 end


[I 2025-06-01 22:28:46,118] Trial 0 finished with value: 478.1224738372816 and parameters: {'hidden_dim': 128, 'dropout': 0.2366351608948103, 'learning_rate': 0.0003521826323511964, 'batch_size': 64, 'k': 8}. Best is trial 0 with value: 478.1224738372816.


Trial 1 params: 256, 0.4423644918657752, 0.0036291877855355284, 8, 12
Trial 1 Epoch 0 start
Trial 1 Epoch 0 end
Trial 1 Epoch 1 start
Trial 1 Epoch 1 end
Trial 1 Epoch 2 start
Trial 1 Epoch 2 end
Trial 1 Epoch 3 start
Trial 1 Epoch 3 end
Trial 1 Epoch 4 start
Trial 1 Epoch 4 end
Trial 1 Epoch 5 start
Trial 1 Epoch 5 end
Trial 1 Epoch 6 start
Trial 1 Epoch 6 end
Trial 1 Epoch 7 start
Trial 1 Epoch 7 end
Trial 1 Epoch 8 start
Trial 1 Epoch 8 end
Trial 1 Epoch 9 start
Trial 1 Epoch 9 end


[I 2025-06-01 22:35:08,933] Trial 1 finished with value: 109.30080866466562 and parameters: {'hidden_dim': 256, 'dropout': 0.4423644918657752, 'learning_rate': 0.0036291877855355284, 'batch_size': 8, 'k': 12}. Best is trial 1 with value: 109.30080866466562.


...

Trial 28 params: 64, 0.3279347983560189, 0.0010417555214566805, 8, 12
Trial 28 Epoch 0 start
Trial 28 Epoch 0 end
Trial 28 Epoch 1 start
Trial 28 Epoch 1 end
Trial 28 Epoch 2 start
Trial 28 Epoch 2 end
Trial 28 Epoch 3 start
Trial 28 Epoch 3 end
Trial 28 Epoch 4 start
Trial 28 Epoch 4 end
Trial 28 Epoch 5 start
Trial 28 Epoch 5 end
Trial 28 Epoch 6 start
Trial 28 Epoch 6 end
Trial 28 Epoch 7 start
Trial 28 Epoch 7 end
Trial 28 Epoch 8 start
Trial 28 Epoch 8 end
Trial 28 Epoch 9 start
Trial 28 Epoch 9 end


[I 2025-06-02 02:10:19,052] Trial 28 finished with value: 127.69209948176895 and parameters: {'hidden_dim': 64, 'dropout': 0.3279347983560189, 'learning_rate': 0.0010417555214566805, 'batch_size': 8, 'k': 12}. Best is trial 14 with value: 98.51181403781062.


Trial 29 params: 128, 0.24876242376318541, 0.002720945829761053, 8, 10
Trial 29 Epoch 0 start
Trial 29 Epoch 0 end
Trial 29 Epoch 1 start
Trial 29 Epoch 1 end
Trial 29 Epoch 2 start
Trial 29 Epoch 2 end
Trial 29 Epoch 3 start
Trial 29 Epoch 3 end
Trial 29 Epoch 4 start
Trial 29 Epoch 4 end
Trial 29 Epoch 5 start
Trial 29 Epoch 5 end
Trial 29 Epoch 6 start
Trial 29 Epoch 6 end
Trial 29 Epoch 7 start
Trial 29 Epoch 7 end
Trial 29 Epoch 8 start
Trial 29 Epoch 8 end
Trial 29 Epoch 9 start
Trial 29 Epoch 9 end


[I 2025-06-02 02:13:38,228] Trial 29 finished with value: 105.4576534043124 and parameters: {'hidden_dim': 128, 'dropout': 0.24876242376318541, 'learning_rate': 0.002720945829761053, 'batch_size': 8, 'k': 10}. Best is trial 14 with value: 98.51181403781062.


Best params: {'hidden_dim': 256, 'dropout': 0.4170134408312916, 'learning_rate': 0.0025668429951594004, 'batch_size': 8, 'k': 10}
Best value: 98.51181403781062


In [33]:
import torch.nn as nn
hidden_dim = study.best_params['hidden_dim']
dropout = study.best_params['dropout']
learning_rate = study.best_params['learning_rate']
batch_size = study.best_params['batch_size']
k = study.best_params['k']

activation = nn.ReLU()

patience = 20
edge_index, edge_weight = build_spatial_graph(spatial_df, k=k)
edge_index = edge_index.to(device)
edge_weight = edge_weight.to(device)
model = SpatioTemporalGNN(
    spatial_features=len(spatial_cols),
    temporal_features=len(temporal_cols),
    hidden_dim=hidden_dim,
    dropout=dropout,
    activation=activation
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
best_val = float('inf')
wait = 0
for epoch in tqdm(range(200)):
    run_epoch(train_batches, model, spatial_x, edge_index, edge_weight, optimizer, batch_size=batch_size)
    val_loss, yv, pv = run_epoch(val_batches, model, spatial_x, edge_index, edge_weight, optimizer=None, batch_size=batch_size, collect_pred=True)
    if val_loss < best_val - 1e-4:
        best_val, wait = val_loss, 0
        torch.save(model.state_dict(), "best_gnn_4.pt")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered!")
            break
    r2 = 1 - np.sum((yv - pv)**2) / np.sum((yv - yv.mean())**2)
    print("Epoch: ", epoch,"Val loss:",val_loss, "R square:", r2)
    


  0%|          | 1/200 [00:46<2:34:52, 46.70s/it]

Epoch:  0 Val loss: 376.3598072260268 R square: 0.90740496


  1%|          | 2/200 [01:40<2:48:10, 50.96s/it]

Epoch:  1 Val loss: 208.58705046421736 R square: 0.9486817


  2%|▏         | 3/200 [02:40<3:00:46, 55.06s/it]

...

Epoch:  60 Val loss: 89.84929077256663 R square: 0.97789454


 31%|███       | 62/200 [56:42<1:38:45, 42.94s/it]

Epoch:  61 Val loss: 82.59672428083596 R square: 0.97967887


 31%|███       | 62/200 [57:25<2:07:49, 55.58s/it]

Early stopping triggered!


In [ ]:
model = SpatioTemporalGNN(
        spatial_features=len(spatial_cols),
        temporal_features=len(temporal_cols),
        hidden_dim=hidden_dim,
        dropout=dropout,
        activation=activation
    )
# load the weights
model.load_state_dict(torch.load("best_gnn_4.pt"))
val_loss, y_true, y_pred = run_epoch(test_batches, model,
                                 spatial_x, edge_index, edge_weight,
                                 optimizer=None, collect_pred=True)



In [ ]:
# calculate adjusted R square
n = len(y_true)  # number of samples
p = len(spatial_cols) + len(temporal_cols)  # number of features
r2 = 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - y_true.mean())**2)
adjusted_r2 = 1 - (1-r2)*(n-1)/(n-p-1)

# calculate MSE
mse = np.mean((y_true - y_pred)**2)

print(f"Adjusted R square: {adjusted_r2:.4f}")
print(f"MSE: {mse:.4f}")

Adjusted R square: 0.9461
MSE: 207.2974
